In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 72.6 MB/s 
     |████████████████████████████████| 895 kB 56.5 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 61 kB 682 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

     |████████████████████████████████| 306 kB 6.7 MB/s 
     |████████████████████████████████| 243 kB 50.0 MB/s 
     |████████████████████████████████| 1.1 MB 61.6 MB/s 
     |████████████████████████████████| 132 kB 74.4 MB/s 
     |████████████████████████████████| 271 kB 71.3 MB/s 
     |████████████████████████████████| 160 kB 77.2 MB/s 
     |████████████████████████████████| 192 kB 70.7 MB/s 


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

url = "https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_"

data_files = {
    "train" : url + "train.tsv",
    "test" : url + "test.tsv"
}

bangla_cricket_dataset = load_dataset("csv", data_files = data_files, sep = "\t")


Using custom data configuration default-f8b1169f8fffa4b2


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/86.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f8b1169f8fffa4b2/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
bangla_cricket_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'class_label'],
        num_rows: 2085
    })
    test: Dataset({
        features: ['id', 'text', 'class_label'],
        num_rows: 380
    })
})

In [5]:
bangla_cricket_dataset["train"][14]

{'class_label': 'negative',
 'id': 227,
 'text': 'সংবাদ সমল্মেনে এসে জাতীয় বীর ম্যাশের সহজ কথা- আমরা খেলেছি বোলিং পিচে কিন্তু তারা খেলেছে ব্যাটিং পিচে।'}

### Initializing tokenizer and datacollator

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "monsoon-nlp/bangla-electra"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer

Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='monsoon-nlp/bangla-electra', vocab_size=29898, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [8]:
data_collator = DataCollatorWithPadding(tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='monsoon-nlp/bangla-electra', vocab_size=29898, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [9]:
def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation = True
  )

tokenized_datasets = bangla_cricket_dataset.map(tokenize_function, batched = True)
tokenized_datasets

  0%|          | 0/3 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'class_label', 'id', 'input_ids', 'text', 'token_type_ids'],
        num_rows: 2085
    })
    test: Dataset({
        features: ['attention_mask', 'class_label', 'id', 'input_ids', 'text', 'token_type_ids'],
        num_rows: 380
    })
})

In [ ]:
tokenized_datasets["train"][0]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'class_label': 'neutral',
 'id': 2760,
 'input_ids': [3, 18913, 2386, 2288, 3728, 35, 4],
 'text': 'আফতাব আপনি ভালো আছেন?',
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0]}

In [ ]:
type(tokenized_datasets)

datasets.dataset_dict.DatasetDict

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.config

loading configuration file https://huggingface.co/monsoon-nlp/bangla-electra/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ed4bba197748161af54c3248a26f97ded9e3509fbefda78285ececa1123fcbe3.c8f2f21ce33891d47310cad825c570e6b77f1cbd2fecde724b1e7bdd2080c0f3
Model config ElectraConfig {
  "_name_or_path": "monsoon-nlp/bangla-electra",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

ElectraConfig {
  "_name_or_path": "monsoon-nlp/bangla-electra",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29898
}

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model, 
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    data_collator = data_collator, 
    tokenizer = tokenizer, 
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, text, class_label.
***** Running training *****
  Num examples = 2085
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 783


KeyError: ignored

In [ ]:
!wget https://www.kaggle.com/furcifer/bangla-newspaper-dataset/download